In [1]:
import numpy as np
import pandas as pd
from faker import Faker
import matplotlib.pyplot as plt

# Set random seed for reproducibility
np.random.seed(0)

# Create a Faker instance to generate American names
fake = Faker()

# Create a list of student names
Students=[]
x=0
while x<100:
    name=fake.first_name()
    if name not in Students:
        Students.append(name)
        x+=1
    else:
        continue
        

# Create a dictionary to store subjects and their respective parameters
subject_params = {
    "Math": (75, 12),       # Mean = 50, Std Deviation = 15
    "Art": (70, 10),        # Mean = 70, Std Deviation = 10
    "Physics": (70, 15),    # Mean = 60, Std Deviation = 12
    "Sport": (80, 8)        # Mean = 80, Std Deviation = 8
}

# Initialize an empty list to store data
data = []

# Generate 100 rows of data
for student in Students:
    for subject, params in subject_params.items():
        # Unpack mean and std deviation from the subject_params dictionary
        mean, std_dev = params
        
        # Generate a random score following a normal distribution for each subject
        score = int(np.random.normal(mean, std_dev))
        
        # Ensure the score is within the range [0, 100]
        score = max(0, min(score, 100))
        
        # Append data to the list
        data.append([student, subject, score])

# Create a DataFrame from the data
df = pd.DataFrame(data, columns=["Student", "Subject", "Score"])

In [ ]:
Student=pd.Series(df['Student'].unique(),name='Student')
Subject=pd.Series(df['Subject'].unique(),name='Subject')
global_vars={'student_name':None,'subject_name':None,'score':None}

In [2]:
#Main menu
def intro():
    print("Welcome!")
    print("I'm here to help teachers manage their student's grades in 4 Subjects: Math,Physics,Sport and Art.")
    print("I already have grades of 100 students, you can add new grades,students and subjects.")
    print("Also I can provide the average score of each student and subject and provide statistics on the student's performance")
    print("in each subject, I can even calculate the top 5 students in all subjects.")
    print("Let's start!")
    main_menu()

def main_menu():
    while True:
        print("What do you want me to do:")
        print("1) Show me the average score in each subject")
        print("2) Show me the avergae score of specific student")
        print("3) Add new grade")
        print("4) Add new subject")
        print("5) Add new student")
        print("6) Correct grade")
        print("7) Provide statistics on specific subject")
        print("8) Show me the top 5 students")
        print("9) Exit")
        next_action=input("Enter the number of the required action: ")
        match next_action:
            case "1":
                avg_per_subject()
            case "2":
                avg_student()
            case "3":
                add_grade()
            case "4":
                add_new_subject()
            case "5":
                add_new_student()
            case "6":
                correct_grade()
            case "7":
                statistics_by_subject()
            case "8":
                top_5_students()
            case "9":
                print("Bye Bye")
                break            

In [3]:
def avg_per_subject():
    while True:
        print(df.groupby('Subject')['Score'].mean())
        next_action= input("Click anything to return to main menu")
        if next_action is not None:
            main_menu()
            break

In [ ]:
def avg_student():
    while True:
        student_name=input('Student name:').title().strip()
        if student_name in Student.values:
            print("{} has average score of {} in {} tests".format(student_name,df.loc[df['Student']==student_name]['Score'].mean(),len(df.loc[df['Student']==student_name])))
        else:
            print("There isn't student called {}".format(student_name,))
        next_action= input("Do you want to check another student's average? (yes/no): ").strip().lower()
        if next_action != 'yes':
            main_menu()
            break

In [ ]:
def add_grade(student_name=None,subject_name=None):
    global df,global_vars
    while True:
        if student_name is None:
            student_name=input("Student name:").title().strip()
            global_vars['student_name']=student_name
        if validation(student_name,Student):
            if subject_name is None:
                subject_name=input("Subject:").title().strip()
                global_vars['subject_name']=subject_name
            if validation(subject_name,Subject):
                if grade_duplication_validation(student_name,subject_name):
                    score=int(input("Score:").strip())
                    global_vars['score']=score
                    if score_validation(score):
                        new_grade=pd.Series({'Student':global_vars['student_name'],'Subject':global_vars['subject_name'],'Score':global_vars['score']})
                        df=pd.concat([df,new_grade.to_frame().T],ignore_index=True)
                        print("we updated that {} gets {} in {}".format(global_vars['student_name'],global_vars['score'],global_vars['subject_name']))
        next_action= input("Do you want to add another grade? (yes/no): ").strip().lower()
        if next_action != 'yes':
            main_menu()
            break

def validation(name,column):
    global global_vars
    if name in column.values:
        print("{}'s validation passed".format(column.name))
        return True
    else:
        print("There isn't {} in the {}'s list".format(name,column.name))
        print("What do you want to do:")
        print("1) Add {} as new {}".format(name,column.name))
        print("2) I want to correct the {}'s name".format(column.name))
        print("3) Return to the main menu")
        next_action=input("Enter the number of the desired action ")
        match next_action:
            case '1':
                if column.name=='Student':
                    add_new_student(name)
                    if validation(name,Student):
                        return True
                else:
                    add_new_subject(name)
                    if validation(name,Subject):
                        return True
            case '2':
                name=input("New {}'s name: ".format(column.name)).title().strip()
                if column.name=="Student":
                    global_vars['student_name']=name
                    add_grade(student_name=global_vars['student_name'])
                else:
                    global_vars['subject_name']=name
                    add_grade(student_name=global_vars['student_name'],subject_name=global_vars['subject_name'])
            case '3':
                main_menu()

def score_validation(score):
    global global_vars
    if score>=0 and score<=100:
        print('score validation passed')
        return True
    else:
        print("The student's score has to be a number between 0 and 100")
        x=0
        while x<3:
            try:
                print("Re-enter the score, you have {} attempts left:".format(3-x))
                x+=1
                score=int(input("Score:").strip())
                global_vars['score']=score
                if score>=0 and score<=100:
                    print('true')
                    return True
                else:
                    print("The student's score has to be a number between 0 and 100")
            except ValueError:
                print("The student's score has to be a number between 0 and 100")
        print("Check again the student's score and try again")
        return False 
    
def grade_duplication_validation(student_name,subject_name):
    global df
    existing_row=df[(df['Student']==student_name)&(df['Subject']==subject_name)]
    if existing_row.shape[0]>0:
        print("{} already has score of {} in {}".format(student_name,int(existing_row['Score']),subject_name))
        print("Do you want to correct the score?")
        if_correct=input("Write 'yes' to correct or anything else to return to main menu").title().strip()
        if if_correct=='Yes':
            df=df[~((df['Student']==student_name)&(df['Subject']==subject_name))]
            return True
        else:
            return False
    else:
        return True

In [ ]:
def add_new_subject(name=None):
    global Subject
    while True:
        if name is None:
            name=input("{}'s Name:".format(Subject.name)).title().strip()

        if name in Subject.values:
            print("There is already {} in the {}'s list".format(name,Subject.name))
            break
        else:
            try:
                if name.isalpha():
                    new_value=pd.Series(name,name=Subject.name)
                    Subject=pd.concat([Subject,new_value],ignore_index=True)
                    print("{} is added to the {}'s list".format(name,Subject.name))
                else:
                    raise ValueError("{}'s Name can only contain letters".format(Subject.name))
            except ValueError as e:
                print(e)
        next_action= input("Do you want to add another subject? (yes/no): ").strip().lower()
        if next_action != 'yes':
            main_menu()
            break

In [4]:
def add_new_student(name=None):
    global Student
    while True:
        if name is None:
            name=input("{}'s Name:".format(Student.name)).title().strip()

        if name in Student.values:
            print("There is already {} in the {}'s list".format(name,Student.name))
            main_menu()
        else:
            try:
                if name.isalpha():
                    new_value=pd.Series(name,name=Student.name)
                    Student=pd.concat([Student,new_value],ignore_index=True)
                    print("{} is added to the {}'s list".format(name,Student.name))
                    main_menu()
                else:
                    raise ValueError("{}'s Name can only contain letters".format(Student.name))
                    add_new_student()
            except ValueError as e:
                print(e)
        next_action= input("Do you want to add another student? (yes/no): ").strip().lower()
        if next_action != 'yes':
            main_menu()
            break

In [6]:
def correct_grade():
    global df,global_vars
    while True:
        student_name=input('Enter student name').title().strip()
        if validation(student_name,Student):
            global_vars['student_name']=student_name
            subject_name=input('Enter subject name').title().strip()
            if validation(subject_name,Subject):
                global_vars['subject_name']=subject_name 
                current_grade=df[(df['Student']==student_name)&(df['Subject']==subject_name)]
                if current_grade.shape[0]>0:
                    print("{} has score of {} in {}, what is the updated score?".format(student_name,int(current_grade['Score']),subject_name))
                    score=int(input("Enter new score").strip())
                    global_vars['score']=score
                    if score_validation(score):
                        df.loc[(df['Student'] == global_vars['student_name']) & (df['Subject'] == global_vars['subject_name']), 'Score'] = global_vars['score']
                        print("we updated that {} gets {} in {}".format(global_vars['student_name'],global_vars['score'],global_vars['subject_name']))
                    next_action= input("Do you want to correct another student's score? (yes/no): ").strip().lower()
                    if next_action != 'yes':
                        main_menu()
                        break
                else:
                    next_action=input("{} doesn't have a score in {}, do you want to enter new score?".format(global_vars['student_name'],global_vars['subject_name'])).strip().lower()
                    if next_action != 'yes':
                        main_menu()
                        break
                    else:
                        add_grade(student_name=global_vars['student_name'],subject_name=global_vars['subject_name'])
                        break

In [11]:
def statistics_by_subject():
    global global_vars
    while True:
        print('For which subject you want to get info on:\n{}'.format('\n'.join(Subject.values)))    
        subject_name=input('Subject name: ').title().strip()
        if subject_name not in Subject.values:
            print("There isn't subject called {}, return to main menu and add {} as subject and add the students' grades".format(subject_name,subject_name))
        else:
            global_vars['subject_name']=subject_name
            grade_historgram(global_vars['subject_name'])
            grade_groups(global_vars['subject_name'])
            general_info(global_vars['subject_name'])
        break
        
def grade_historgram(subject_name):
    df[df['Subject']==subject_name]['Score'].plot(kind='hist',title="{}'s' grade distribution".format(subject_name))
    plt.show()

def grade_groups(subject_name):
    bins=[0,50,70,80,90,101]
    labels=['<50','51-70','71-80','81-90','91-100']
    
    scores = df[df['Subject'] == subject_name]
    result = scores.groupby(pd.cut(scores['Score'], bins=bins, labels=labels)).size()
    print('Grades distribution by groups:')
    for index, row in result.to_frame().iterrows():
        print("{}: {}".format(index,row.values[0]))

def general_info(subject_name):
    info=round(df[df['Subject']==subject_name].describe(),2)
    info.index=['Number of tests:','Average:','Std:','Lowest score:','25% score','50% score','75% score','Highest score']
    print(info)

In [12]:
def top_5_students():
    while True:
        avg=df.groupby('Subject')['Score'].mean()
        std=df.groupby('Subject')['Score'].std()
        data=[]
        for student in Student.values:
            for subject in Subject.values:
                Zscore=z_score(student,subject,avg,std)
                class_rank=rank_in_class(student,subject)
                data.append([student,subject,Zscore,class_rank])
        df_z_score=pd.DataFrame(data=data,columns=['Student','Subject','Z-score','class_rank'])
        df_z_score=total_z_score(df_z_score)
        print("Here are the top 5 students based on Z-score calculation and their class' rank in each subject")
        print("(For example: position 8 in Art means that this student got the 8th highest grade in Art)\n")
        print(df_z_score.head())
        break
    

def z_score(student,subject,avg,std):
    student_grade=int(df[(df['Student']==student)&(df['Subject']==subject)]['Score'])
    Zscore=round(float(student_grade-avg[subject])/std[subject],2)
    return Zscore

def rank_in_class(student,subject):
    subject_grades=sorted(df[df['Subject']==subject]['Score'].to_list(),reverse=True)
    rank=subject_grades.index(df[(df['Student']==student)&(df['Subject']==subject)]['Score'].values[0])+1
    return rank

def total_z_score(df_z_score):
    total_z_score=df_z_score.groupby('Student')['Z-score'].sum().reset_index()
    total_z_score.columns=['Student','Total Z-score']
    df_z_score=df_z_score.merge(total_z_score,on='Student')
    df_z_score=df_z_score.pivot(columns='Subject',index=['Student','Total Z-score'],values='class_rank').sort_values(by='Total Z-score',ascending=False).reset_index()
    return df_z_score

In [14]:
intro()

Welcome!
I'm here to help teachers manage their student's grades in 4 Subjects: Math,Physics,Sport and Art.
I already have grades of 100 students, you can add new grades,students and subjects.
Also I can provide the average score of each student and subject and provide statistics on the student's performance
in each subject, I can even calculate the top 5 students in all subjects.
Let's start!
What do you want me to do:
1) Show me the average score in each subject
2) Show me the avergae score of specific student
3) Add new grade
4) Add new subject
5) Add new student
6) Correct grade
7) Provide statistics on specific subject
8) Show me the top 5 students
9) Exit
Enter the number of the required action: 9
Bye Bye
